In [113]:
import psycopg2
import psycopg2.extras
import pandas as  pd
from db_credentials import postgres_config

cur = None
conn = None

try:
    conn = psycopg2.connect(host=postgres_config['hostname'],
                dbname=postgres_config['database'],
                user=postgres_config['username'],
                password=postgres_config['pwd'],
                port=postgres_config['port_id']
            )
    def extract():
        cur = conn.cursor(cursor_factory=psycopg2.extras.DictCursor)

        extract_data = '''select store, dayofweek, "Date", customers from public.store'''

        cur.execute(extract_data)
    
        data = cur.fetchall()
        data = pd.DataFrame(data, columns=['store', 'day_of_week', 'date', 'customers'])
        
        return data
    
    def transform():
        
        # change type
        data['date'] = pd.to_datetime(data['date'])
        data['store'] = data['store'].astype('int64')
        data['day_of_week'] = data['day_of_week'].astype('int64')
        data['customers'] = data['customers'].astype('int64')

        # fill out na
        data.dropna(axis=0, inplace=True)
        
        return data
    
    def data_filtering():
        
        data_filtered = data[['date', 'store', 'day_of_week', 'customers']].groupby(['store', 'date']).sum().reset_index()
        return data_filtered
    
    def load():
        
        create_script = '''
        CREATE TABLE IF NOT EXISTS public.customers'''
        
        cur.execute(create_script)
        cur.commit()
        
        # insert DataFrame into database
        data_filtered.to_sql('customers', conn)
        print('rows inserted: {data_filtered.shape[0]}')
        print('etl process done')      
            
        
except Exception as error:
    print(error)
finally:
    if conn is not None:
        conn.close()
    if cur is not None:
        cur.close()

In [114]:
_ = load()

AttributeError: 'NoneType' object has no attribute 'execute'

In [112]:
_

''